In [1]:
import fasttext
import fasttext.util
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

In [2]:
model = fasttext.load_model("../../Downloads/ubertext.fiction_news_wikipedia.filter_rus+short.tokens.txt.algo-cbow.epochs-15.subwords-2..5.neg_sampling-15.bin")


In [3]:
tsn_csv = "tsn_articles.csv"
ukrinform_csv = "ukrinform.csv"
pravda_csv = "pravda_articles.csv"
strana_csv = "0strana_articles.csv"
_112_csv = "112.csv"
newsone_csv = "newsone.csv"

true_news = [tsn_csv, pravda_csv, ukrinform_csv]
prop_news = [strana_csv, _112_csv, newsone_csv]

dr = "../news/csv/"

In [4]:
tsn = pd.read_csv(dr + tsn_csv, delimiter="\t", index_col=0)
pravda = pd.read_csv(dr + pravda_csv, delimiter="\t", index_col=0)
strana = pd.read_csv(dr + strana_csv, delimiter="\t", index_col=0)
_112 = pd.read_csv(dr + _112_csv, delimiter="\t", index_col=0)
newsone = pd.read_csv(dr + newsone_csv, delimiter="\t", index_col=0)
ukrinform = pd.read_csv(dr + ukrinform_csv, delimiter="\t", index_col=0)

In [5]:
strana.head()

,Article
number,
0,Російські нафта і газ підтримують позитивний б...
1,Фінансування Мінкульту не буде переглянуто у р...
2,Соратниця Санду пішла з виборів через скандал ...
3,"Супутникова система, що прибула в Україну. Фот..."
4,"контейнеровоз, що сів на мілину, шість днів бл..."


In [37]:
true_vectors = []
false_vectors = []

# df = pd.DataFrame()

for i, news in enumerate(true_news):
    true_content = pd.read_csv(dr + news, delimiter="\t")
#     if i < len(true_news)-1:
#         df[i] = true_content[" Article"].apply(lambda x: model.get_sentence_vector(str(x)))
#     else:
#         df[i] = true_content["Article"].apply(lambda x: model.get_sentence_vector(str(x)))
        
#     break
#     df = pd.concat([df, t])
    for t in true_content.iterrows():
        if i < len(true_news)-1:
            if pd.isna(t[1][" Article"]):
                continue
            true_vector = model.get_sentence_vector(t[1][" Article"])
        else:
            if pd.isna(t[1]["Article"]):
                continue
            true_vector = model.get_sentence_vector(t[1]["Article"])
        true_vectors.append(true_vector)
        
for i, news in enumerate(prop_news):
    false_content = pd.read_csv(dr + news, delimiter="\t")
    for f in false_content.iterrows():
        false_vector = model.get_sentence_vector(f[1]["Article"])
        false_vectors.append(false_vector)

# df

In [39]:
len(true_vectors), len(false_vectors)

(128587, 16417)

In [38]:
df = pd.DataFrame(true_vectors + false_vectors)
df['y'] = [0]*len(true_vectors) + [1]*len(false_vectors)
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,y
0,0.045944,0.033212,-0.019494,0.017835,0.017329,0.044720,0.036844,-0.013484,0.021245,0.006724,...,0.025386,0.011930,0.012249,-0.077265,0.017796,0.028253,0.015929,0.006496,-0.002924,0
1,0.059813,0.036721,-0.018274,0.032621,0.015537,0.051469,0.053786,-0.021347,0.017381,-0.009109,...,0.022771,0.015956,0.019941,-0.073319,0.021163,0.038026,0.010041,-0.003658,-0.010412,0
2,0.057720,0.031666,-0.015856,0.018313,0.018238,0.038563,0.043197,-0.020581,0.024852,-0.007280,...,0.020274,0.016442,0.019216,-0.061398,0.024590,0.027343,0.011330,-0.011836,0.008885,0
3,0.055922,0.037115,-0.009455,0.030506,0.023757,0.047180,0.052907,-0.034172,0.021569,-0.008985,...,0.020767,0.021013,0.023593,-0.070772,0.012401,0.030589,-0.000591,-0.005169,-0.008243,0
4,0.055116,0.015518,-0.011605,0.043243,0.007095,0.042123,0.033189,-0.015345,0.020922,-0.022534,...,0.020315,0.002096,0.032026,-0.066096,0.002333,0.044644,0.008299,-0.004881,-0.010561,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144999,0.057816,0.030768,-0.017235,0.026246,0.021929,0.042852,0.046335,-0.022112,0.019113,-0.003631,...,0.030071,0.014059,0.017375,-0.067576,0.025667,0.042140,-0.001227,0.004661,-0.006640,1
145000,0.053990,0.032169,-0.017542,0.023458,0.020964,0.043306,0.043331,-0.019090,0.019162,-0.002829,...,0.030633,0.011633,0.017338,-0.068067,0.022882,0.040825,0.000284,0.005336,-0.006218,1
145001,0.055371,0.029537,-0.017212,0.021284,0.018991,0.044607,0.042242,-0.019062,0.018636,-0.003438,...,0.030407,0.011757,0.016129,-0.067525,0.020673,0.039090,0.001660,0.002860,-0.005084,1
145002,0.058424,0.026798,-0.017176,0.010653,0.023448,0.047812,0.047891,-0.018901,0.022463,-0.004640,...,0.026067,0.015758,0.020223,-0.073846,0.022762,0.031064,0.004096,-0.001435,-0.004549,1


In [40]:
X, y = df.iloc[:, :-1], df['y']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

clf = GaussianNB().fit(X_train, y_train)
prediction = clf.predict(X_test)

In [41]:
y = y_test.to_numpy()

conf = confusion_matrix(prediction, y)
acc = accuracy_score(prediction, y)
precision = precision_score(prediction, y, zero_division=0)
recall = recall_score(prediction, y, zero_division=0)
f1 = f1_score(prediction, y, zero_division=0)

In [42]:
print("Confusion matrix:\n", conf)
print("Accuracy: ", acc)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

Confusion matrix:
 [[17849   891]
 [ 7851  2410]]
Accuracy:  0.6985621185476363
Precision:  0.7300817933959406
Recall:  0.23486989572166456
F1:  0.3554048075505088
